In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

def plot_energy_action_heatmaps(df=None, file_path=None, save=True):
    """
    Plots heatmaps of energy actions vs SOC and Hour.
    Automatically detects action columns and bins SOC/hour for clarity.
    """
    if df is None and file_path is None:
        raise ValueError("Either df or file_path must be provided.")
    if df is None:
        df = pd.read_csv(file_path)

    if "soc" not in df.columns or "hour" not in df.columns:
        raise ValueError("DataFrame must contain 'soc' and 'hour' columns.")

    df["soc_bin"] = pd.cut(df["soc"], bins=np.linspace(0, 1, 21), include_lowest=True)
    df["hour_bin"] = df["hour"].round().astype(int)

    action_cols = [c for c in df.columns if c not in ["soc", "hour", "soc_bin", "hour_bin"]]

    for action in action_cols:
        pivot = df.groupby(["soc_bin", "hour_bin"])[action].mean().unstack()
        plt.figure(figsize=(10, 6))
        cmap = "coolwarm" if "batt" in action else "YlOrBr"
        center = 0 if "batt" in action else None
        sns.heatmap(pivot, cmap=cmap, center=center)
        plt.title(f"{action} vs SOC and Hour")
        plt.xlabel("Hour of Day")
        plt.ylabel("SOC Bin")
        plt.tight_layout()
        if save and file_path:
            plt.savefig(f'{Path(file_path).stem}_{action}.png')
        plt.show()


In [ ]:
df = pd.read_csv("example_policy_log.csv")
plot_energy_action_heatmaps(df=df)